# Automated ML

This Jupyter Notebook is a portion of a larger project, it includes an AutoML experiment and model deployment. Other portions of the project include Hyperparameter tuning through HyperDrive for an experiment and project reference information.

In [1]:
import csv
import logging
import os

import joblib
import numpy as np
import pandas as pd
import pickle
import pkg_resources

import azureml.core
from azureml.core import Dataset, Experiment, Workspace
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace

Initialize workspace from config

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AQRQYJRAY to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-138219
aml-quickstarts-138219
southcentralus
9e65f93e-bdd8-437b-b1e8-0647cd6098f7


## Create Azure ML Experiment

Create and name the experiment

In [3]:
# Choose a name for experiment
experiment_name = 'automl-heart-failure'
project_folder = './heartfailure'

experiment = Experiment(ws, experiment_name)

## Create or Attach an Auto ML Compute Cluster
Search for existing compute cluster; if not found, create one

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# If using a different existing cluster, enter the name in place of 'mlecscompute' below:
amlcompute_cluster_name = 'mlecscompute'

# Verify cluster existence: 
try: 
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found; creating new cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_v2', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it.

Running


## Dataset
Search for uploaded dataset in AzureML Studio; if not found, find it through the URL

## Acknowledgements
This Jupyter Notebook utilizes the Heart Failure Prediction Dataset, downloaded from Kaggle under License "Attribution 4.0 International" or "CC BY 4.0." Credit: Davide Chicco, Giuseppe Jurman: Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making 20, 16 (2020).

### Kaggle Dataset Description
Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

### Dataset Column Details
 - Age (patient age, in years)
 - Anaemia (binary classification)
 - Creatinine Phosphokinase (mcg/L of CPK enzyme in patient's blood)
 - Diabetes (binary classification)
 - Ejection fraction (% of blood exiting heart with each contraction)
 - High blood pressure (binary classification)
 - Platelets (kiloplatelets/mL in patient's blood)
 - Serum creatinine (md/dL of creatinine in patient's blood)
 - Serum sodium (mEq/L of sodium in patient's blood)
 - Sex (binary classification)
 - Smoking (binary classification)
 - Time (days included in follow-up period) 
 
 TARGET COLUMN:
 - Death event (binary classification; if patient died during follow-up period)

### Task
Create a binary classification model to assess the likelihood of a patient's death by heart failure event.

In [5]:
# Attempt to load the dataset from the Workspace, but otherwise, create from file
# dataset file located at 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
found = False
# key should be set to 'heartfailuredataset' 
key = 'heartfailuredataset'
# description_text should be set to 'heartfailuredataset' 
description_text = 'heartfailuredataset'

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    datasetfile = 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(datasetfile)
    dataset = dataset.register(workspace = ws, name = key, description = description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# AutoML settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# AutoML config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# Submit experiment
automl_run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on mlecscompute with default configuration
Running on remote compute: mlecscompute
Parent Run ID: AutoML_72afd690-c85b-417b-96a2-acdcc5baba1a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were a

## Run Details

`RunDetails` widget shows the different experiments.

In [8]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_automl_run, fitted_automl_model = automl_run.get_output()
print(best_automl_run)
print(fitted_automl_model)

best_automl_run.get_metrics()

joblib.dump(fitted_automl_model, 'automlheartmodel.sav')

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automl-heart-failure,
Id: AutoML_72afd690-c85b-417b-96a2-acdcc5baba1a_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_alpha=0.3157894736842105,
                                                                                                reg_lambda=0.631578947368421,
      

['automlheartmodel.sav']

In [10]:
best_automl_run.get_details()

{'runId': 'AutoML_72afd690-c85b-417b-96a2-acdcc5baba1a_38',
 'target': 'mlecscompute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-09T02:22:21.952217Z',
 'endTimeUtc': '2021-02-09T02:24:56.322691Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl-heart-failure\',\'compute_target\':\'mlecscompute\',\'subscription_id\':\'9e65f93e-bdd8-437b-b1e8-0647cd6098f7\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_72afd690-c85b-417b-96a2-acdcc5baba1a_38","experiment_name":"automl-heart-failure","workspace_name":"quick-starts-ws-138219","subscription_id":"9e65f93e-bd

## Model Deployment

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
#Deployment dependencies
import json
import requests

from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [12]:
# Register best model
registered_heart_model = Model.register(workspace = ws, model_path = './automlheartmodel.sav', model_name = 'regheartmodel.sav')

Registering model regheartmodel.sav


In [13]:
# Create inference config
env = Environment.get(workspace = ws, name = 'AzureML-AutoML')
infer_config = InferenceConfig(entry_script = 'score.py', environment = env)

In [14]:
# Deploy model
deployheartmodel = AciWebservice.deploy_configuration()
svcs = Model.deploy(workspace = ws, name = 'deploy', models = [registered_heart_model], inference_config = infer_config, deployment_config = deployheartmodel)
svcs.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(svcs.state)

Healthy


In [16]:
print(svcs.scoring_uri)

http://fbbc793c-f955-4471-a2af-6e0f7f70bc56.southcentralus.azurecontainer.io/score


## Test Web service

Send a request to the the deployed web service to test it.

In [17]:
# Web service URL
# http://36f3327a-e9dd-420b-a72e-4e03e8bef587.southcentralus.azurecontainer.io/score
scoring_uri = svcs.scoring_uri

data = {'data':[{
    'age' : 75, 
    'anaemia' : 0, 
    'creatinine_phosphokinase' : 582,
    'diabetes' : 0,
    'ejection_fraction' : 20,
    'high_blood_pressure' : 1,
    'platelets' : 265000,
    'serum_creatinine' : 1.9, 
    'serum_sodium' : 130,
    'sex' : 1,
    'smoking' : 0,
    'time' : 4
}]}

# Convert to json string
input_data = json.dumps(data)

# Content type
headers = {'Content-Type': 'application/json'}

In [18]:
# Make request and display response
result = requests.post(scoring_uri, input_data, headers = headers)
print('prediction:', result.text)

prediction: [1]


In [19]:
env.save_to_directory('./training', overwrite = False)

## Clean-up

Print web service and clean-up deployed resources

In [20]:
# Print web service logs
print(svcs.get_logs())

2021-02-09T02:29:41,166517702+00:00 - gunicorn/run 
2021-02-09T02:29:41,166735499+00:00 - rsyslog/run 
2021-02-09T02:29:41,266542280+00:00 - iot-server/run 
2021-02-09T02:29:41,266661678+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [21]:
# Delete the service
svcs.delete()

In [22]:
# Shut down compute
compute_target.delete()